In [2]:
import pandas as pd
import seaborn as sns

## Reading data 
Final CSV files generated by *merge_data.py*

In [3]:
df_p1 = pd.read_csv("C:/Users/marim/Documents/Faculdade/TCC/patient_act_data_CSV/1379-38_data_final.csv")

In [10]:
df_p1.head()

,Date,Time,X,Y,Z,T,Sleep,activity_index
0,2018-07-27,15:55:55,-0.0359,-0.2505,-1.0169,36.2,0,0.157358
1,2018-07-27,15:55:55,-0.1655,-0.1098,-0.8518,36.2,0,0.157358
2,2018-07-27,15:55:55,-0.1026,-0.1528,-0.9203,36.2,0,0.157358
3,2018-07-27,15:55:55,-0.0084,-0.1958,-0.9806,36.2,0,0.157358
4,2018-07-27,15:55:55,-0.0045,-0.2154,-0.9243,36.2,0,0.157358


In [15]:
X = df_p1.drop(['Date', 'Time', 'Sleep'], axis=1)
y = df_p1['Sleep']

In [5]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [7]:
from sklearn.ensemble import RandomForestRegressor

In [17]:
rfr = RandomForestRegressor(random_state=11)

In [18]:
rfr.fit(X_train, y_train)

RandomForestRegressor(random_state=11)

In [19]:
y_pred = rfr.predict(X_test)

In [20]:
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score

In [21]:
mean_absolute_error(y_pred, y_test)

np.float64(0.0029244269998778788)

In [22]:
root_mean_squared_error(y_pred, y_test)

np.float64(0.03428255567157017)

In [23]:
r2_score(y_pred, y_test)

0.994297211214533

In [25]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn+fp)
print(specificity)

ValueError: Classification metrics can't handle a mix of binary and continuous targets